##CV features

In [4]:
!pip install advertools

In [5]:
!pip install ntlk

ERROR: Could not find a version that satisfies the requirement ntlk (from versions: none)
ERROR: No matching distribution found for ntlk


In [6]:
import re
import logging 
import gensim
import datetime
import numpy as np
import pandas as pd
%matplotlib inline
import seaborn as sns
import xgboost as xgb 
from time import time 
import pandas as pd
import time
import nltk
from nltk.corpus import stopwords
# Config
import advertools as adv
from tqdm import tqdm 
from utils import metrics
from functools import reduce
from utils import featureScore
from utils import folderPath
from textblob import TextBlob
from sklearn.manifold import TSNE
from collections import defaultdict
from gensim.models import FastText
from sklearn.metrics import r2_score
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from typing import List, Callable, Dict
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from utils import removeColumnContainString
from sklearn.model_selection import RandomizedSearchCV
from utils import hyperparameterTuning_RandomForest
from utils import hyperparameterTuning_XGBoost
from utils import hyperparameterTuning_MLP
from utils import plot_feature_importance
from sklearn.model_selection import train_test_split
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
df = pd.read_csv('/content/drive/MyDrive/NLP/CvFeatures.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'image_type', 'account', 'caption',
       'profile_name', 'profile_image_link', 'biography', 'id', 'external_url',
       'following', 'likes', 'media_type', 'posts_count', 'followers',
       'is_verified', 'datetime', 'image_url.1', 'url', 'comments',
       'input_account', 'input_posts_number', 'input_date_of_post',
       'input_post_type', 'input_stop_id', 'warning', 'error', 'tags',
       'confidence_score', 'accent_color', 'is_bw', 'dominant_colors',
       'bg_color', 'fore_color', 'industry', '_category', 'region'],
      dtype='object')

In [10]:
df

,Unnamed: 0,Unnamed: 0.1,image_type,account,caption,profile_name,profile_image_link,biography,id,external_url,...,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color,industry,_category,region
0,0,0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,Yeast Bakery,https://instagram.fman2-1.fna.fbcdn.net/v/t51....,We are now open at our new site: Unit 1 Canal ...,2.620000e+18,http://www.yeastbakery.com/,...,"['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...","[0.788235294117647, 0.00784313725490196, 0.007...",0,"[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]",hospitality,bakery,UK
1,1,1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,Yeast Bakery,https://instagram.fman2-1.fna.fbcdn.net/v/t51....,We are now open at our new site: Unit 1 Canal ...,2.790000e+18,http://www.yeastbakery.com/,...,"['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]",hospitality,bakery,UK
2,2,2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,Yeast Bakery,https://instagram.fman2-1.fna.fbcdn.net/v/t51....,We are now open at our new site: Unit 1 Canal ...,2.690000e+18,http://www.yeastbakery.com/,...,"['food', 'fruit', 'baked goods', 'breakfast', ...","[0.9808361530303955, 0.9546540379524231, 0.947...","[0.6588235294117647, 0.4392156862745098, 0.141...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",hospitality,bakery,UK
3,3,3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",Yeast Bakery,https://instagram.fman2-1.fna.fbcdn.net/v/t51....,We are now open at our new site: Unit 1 Canal ...,2.670000e+18,http://www.yeastbakery.com/,...,"['food', 'bread', 'fast food', 'bun', 'america...","[0.9958561658859253, 0.981575608253479, 0.9799...","[0.24313725490196078, 0.12549019607843137, 0.0...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",hospitality,bakery,UK
4,4,4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,Yeast Bakery,https://instagram.fman2-1.fna.fbcdn.net/v/t51....,We are now open at our new site: Unit 1 Canal ...,2.620000e+18,http://www.yeastbakery.com/,...,"['text', 'font', 'design', 'yellow', 'graphics...","[0.9993002414703369, 0.9640201926231384, 0.951...","[0.792156862745098, 0.7372549019607844, 0.0039...",0,"[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]",hospitality,bakery,UK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,88511,88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,Fishmongers/ButchersTwickenham,https://scontent-lcy1-1.cdninstagram.com/v/t51...,"Local Family Fishmongers & Butchers , fresh fi...",2.700000e+18,http://sandysfish.net/,...,"['text', 'food', 'snack']","[0.9999812841415405, 0.9580191373825073, 0.915...","[0.6588235294117647, 0.00784313725490196, 0.71...",0,"[0.5019607843137255, 0.0, 0.5019607843137255]","[0.5019607843137255, 0.0, 0.5019607843137255]","[0.5019607843137255, 0.5019607843137255, 0.501...",retail,fishmongers / butchers,UK
88512,88512,88512,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day one brochure tour !! #sandysfishmongers #b...,Fishmongers/ButchersTwickenham,https://scontent-lcy1-1.cdninstagram.com/v/t51...,"Local Family Fishmongers & Butchers , fresh fi...",2.700000e+18,http://sandysfish.ne

In [11]:
df.drop(df.columns.difference(['account','profile_name','biography','image_type','caption','comments','tags','following','confidence_score','accent_color','is_bw','dominant_colors','bg_color','fore_color','posts_count','followers','tags','confidence_score','accent_color','is_bw','dominant_colors','bg_color','fore_color','industry','datetime','likes']), 1, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [12]:
df

,image_type,account,caption,profile_name,biography,following,likes,posts_count,followers,datetime,comments,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color,industry
0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,29.0,280.0,8702.0,1.626600e+12,0.0,"['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...","[0.788235294117647, 0.00784313725490196, 0.007...",0,"[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]",hospitality
1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,44.0,280.0,8702.0,1.646920e+12,0.0,"['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]",hospitality
2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,209.0,280.0,8702.0,1.634730e+12,9.0,"['food', 'fruit', 'baked goods', 'breakfast', ...","[0.9808361530303955, 0.9546540379524231, 0.947...","[0.6588235294117647, 0.4392156862745098, 0.141...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",hospitality
3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,95.0,280.0,8702.0,1.633090e+12,3.0,"['food', 'bread', 'fast food', 'bun', 'america...","[0.9958561658859253, 0.981575608253479, 0.9799...","[0.24313725490196078, 0.12549019607843137, 0.0...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",hospitality
4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,101.0,280.0,8702.0,1.626930e+12,6.0,"['text', 'font', 'design', 'yellow', 'graphics...","[0.9993002414703369, 0.9640201926231384, 0.951...","[0.792156862745098, 0.7372549019607844, 0.0039...",0,"[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]",hospitality
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,8.0,1863.0,2321.0,1.636550e+12,0.0,"['text', 'food', 'snack']","[0.9999812841415405, 0.9580191373825073, 0.915...","[0.6588235294117647, 0.00784313725490196, 0.71...",0,"[0.5019607843137255, 0.0, 0.5019607843137255]","[0.5019607843137255, 0.0, 0.5019607843137255]","[0.5019607843137255, 0.5019607843137255, 0.501...",retail
88512,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day one brochure tour !! #sandysfishmongers #b...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,9.0,1863.0,2321.0,1.636550e+12,0.0,"['text', 'poster', 'cartoon']","[0.9999863505363464, 0.9184243679046631, 0.910...","[0.01568627450980392, 0.5098039215686274, 0.78...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]",retail
88513,"['Not Clip', 'Not LineDraw']",sandysfishmongers,"A unique amber coloured blue veined cheese , w...",Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,11.0,1863.0,2321.0,1.636540e+12,2.0,"['text', 'yellow', 'packaging and labeling', '...","[0.9997624158859253, 0.9444

In [13]:
data = df[df['account'].notna()]
data

,image_type,account,caption,profile_name,biography,following,likes,posts_count,followers,datetime,comments,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color,industry
0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,29.0,280.0,8702.0,1.626600e+12,0.0,"['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...","[0.788235294117647, 0.00784313725490196, 0.007...",0,"[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]",hospitality
1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,44.0,280.0,8702.0,1.646920e+12,0.0,"['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]",hospitality
2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,209.0,280.0,8702.0,1.634730e+12,9.0,"['food', 'fruit', 'baked goods', 'breakfast', ...","[0.9808361530303955, 0.9546540379524231, 0.947...","[0.6588235294117647, 0.4392156862745098, 0.141...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",hospitality
3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,95.0,280.0,8702.0,1.633090e+12,3.0,"['food', 'bread', 'fast food', 'bun', 'america...","[0.9958561658859253, 0.981575608253479, 0.9799...","[0.24313725490196078, 0.12549019607843137, 0.0...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",hospitality
4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,101.0,280.0,8702.0,1.626930e+12,6.0,"['text', 'font', 'design', 'yellow', 'graphics...","[0.9993002414703369, 0.9640201926231384, 0.951...","[0.792156862745098, 0.7372549019607844, 0.0039...",0,"[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]",hospitality
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day 2 Christmas brochure tour Day one brochure...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,10.0,1863.0,2321.0,1.636710e+12,1.0,"['text', 'book', 'screenshot', 'brochure', 'pr...","[0.9999996423721313, 0.9184167385101318, 0.910...","[0.011764705882352941, 0.5019607843137255, 0.7...",0,"[1.0, 1.0, 1.0]","[0.0, 0.0, 1.0]","[1.0, 1.0, 1.0]",retail
88510,"['Not Clip', 'Not LineDraw']",sandysfishmongers,#ArmisticeDay #poppy #lestwerespect,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,50.0,1863.0,2321.0,1.636620e+12,0.0,"['building', 'text', 'electronic signage', 'ch...","[0.9849988222122192, 0.9520490169525146, 0.921...","[0.6470588235294118, 0.1568627450980392, 0.149...",0,"[0.5019607843137255, 0.5019607843137255, 0.501...","[0.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",retail
88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,8.0,1863.0,2321.0,1.636550e+12,0.0,"['text', 'food', 'snack']","[0.9999812841415405, 0.9580191373825073, 0.915...","[0.6588235294117647, 0.0078431

In [14]:
data.to_csv('CV.csv')

In [15]:
# df=pd.read_csv('/content/drive/MyDrive/NLP/CV.csv')

In [16]:
# df

In [17]:
data['caption_length'] = data['caption'].str.len()
data['biography_length'] = data['biography'].str.len()
# df['caption_upper'] = df['caption'].str.findall(r'[A-Z]').str.len()
data['po_co'] = data['posts_count']/(data['comments']+1)
data['pof'] = data['posts_count']/(data['following']+1)
data['user_count'] = data.groupby('account')['posts_count'].transform('count')
data['profile_name_len'] = data['profile_name'].str.len()
data['fol'] = data['followers']/(data['following']+1)
data['act'] = data['comments']/(data['followers']+1)
data['pos'] = data['posts_count']/(data['followers']+1)
data['comments_max'] = data.groupby('account')['comments'].transform('max')
data['comments_min'] = data.groupby('account')['comments'].transform('min')
data['comments_mean'] = data.groupby('account')['comments'].transform('mean')
data['comments_std'] = data.groupby('account')['comments'].transform('std')
data['comments_followers']=data['comments']*data['followers']
data['followers_comments_mean']=data['comments_mean']*data['followers']
data['fol2']=data['fol']*data['fol']
data['fol_pos']=data['fol']*data['pos']
data['fol_pow'] = data['followers']*(data['following'])
data['po_co_pow'] = data['posts_count']*(data['comments'])
data['comments_mean_diff'] = data['comments'] / (data['comments_mean']+1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [18]:
data

,image_type,account,caption,profile_name,biography,following,likes,posts_count,followers,datetime,...,comments_min,comments_mean,comments_std,comments_followers,followers_comments_mean,fol2,fol_pos,fol_pow,po_co_pow,comments_mean_diff
0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,29.0,280.0,8702.0,1.626600e+12,...,0.0,5.85000,8.949624,0.0,50906.700000,607.699315,0.793110,3063104.0,0.0,0.000000
1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,44.0,280.0,8702.0,1.646920e+12,...,0.0,5.85000,8.949624,0.0,50906.700000,607.699315,0.793110,3063104.0,0.0,0.000000
2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,209.0,280.0,8702.0,1.634730e+12,...,0.0,5.85000,8.949624,78318.0,50906.700000,607.699315,0.793110,3063104.0,2520.0,1.313869
3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,95.0,280.0,8702.0,1.633090e+12,...,0.0,5.85000,8.949624,26106.0,50906.700000,607.699315,0.793110,3063104.0,840.0,0.437956
4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,101.0,280.0,8702.0,1.626930e+12,...,0.0,5.85000,8.949624,52212.0,50906.700000,607.699315,0.793110,3063104.0,1680.0,0.875912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day 2 Christmas brochure tour Day one brochure...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,10.0,1863.0,2321.0,1.636710e+12,...,0.0,0.81982,1.063427,2321.0,1902.801802,16.013796,3.210686,1343859.0,1863.0,0.549505
88510,"['Not Clip', 'Not LineDraw']",sandysfishmongers,#ArmisticeDay #poppy #lestwerespect,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,50.0,1863.0,2321.0,1.636620e+12,...,0.0,0.81982,1.063427,0.0,1902.801802,16.013796,3.210686,1343859.0,0.0,0.000000
88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,8.0,1863.0,2321.0,1.636550e+12,...,0.0,0.81982,1.063427,0.0,1902.801802,16.013796,3.210686,1343859.0,0.0,0.000000
88512,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day one brochure tour !! #sandysfishmongers #b...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,9.0,1863.0,2321.0,1.636550e+12,...,0.0,0.81982,1.063427,0.0,1902.801802,16.013796,3.210686,1343859.0,0.0,0.000000


In [19]:
len(data.account.unique())

981

In [20]:
# df.drop(['account'], axis = 1,inplace=True)
# df.drop(['caption'], axis = 1,inplace=True)

data.drop(['profile_name'], axis = 1,inplace=True)
data.drop(['biography'], axis = 1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [21]:
#calculating missing values in the dataset
def missing_Values(data):
  missing_values = data.isnull().sum()
  missing_per = (missing_values/data.shape[0])*100
  missing_table = pd.concat([missing_values,missing_per], axis=1, ignore_index=True) 
  missing_table.rename(columns={0:'Total Missing Values',1:'Missing %'}, inplace=True)
  return missing_table

missing_Values(data)

,Total Missing Values,Missing %
image_type,0,0.000000
account,0,0.000000
caption,330,0.372822
following,0,0.000000
likes,0,0.000000
posts_count,0,0.000000
followers,0,0.000000
datetime,0,0.000000
comments,0,0.000000
tags,0,0.000000


In [22]:
#fill the missing values with zero

data['likes'] = data['likes'].fillna(0)
data['biography_length'] = data['biography_length'].fillna(0)
data['profile_name_len'] = data['profile_name_len'].fillna(0)
data['caption_length'] = data['caption_length'].fillna(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [23]:
#calculating missing values in the dataset
missing_Values(data)

,Total Missing Values,Missing %
image_type,0,0.000000
account,0,0.000000
caption,330,0.372822
following,0,0.000000
likes,0,0.000000
posts_count,0,0.000000
followers,0,0.000000
datetime,0,0.000000
comments,0,0.000000
tags,0,0.000000


## MetaFeatures

In [24]:
df1=data

In [25]:
df1 = df[df['datetime'].notna()]

In [26]:
import datetime
def timeStampConversion(x):
    return datetime.datetime.fromtimestamp(int(x)/1000)

df1['datetime'] = pd.to_numeric(df1['datetime'], downcast="float")
df1['datetime'] = df1['datetime'].apply(lambda x: timeStampConversion(x)) 
df1["datetime"] = pd.to_datetime(df1["datetime"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [27]:
df1['Dow'] = df1['datetime'].dt.weekday
df1['Hod']= df1['datetime'].dt.hour
df1['Date']= df1['datetime'].dt.date

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [28]:
df1['mon']=(df1['Dow']==0).astype(int)
df1['tue']=(df1['Dow']==1).astype(int)
df1['wed']=(df1['Dow']==2).astype(int)
df1['thu']=(df1['Dow']==3).astype(int)
df1['fri']=(df1['Dow']==4).astype(int)  
df1['sat']=(df1['Dow']==5).astype(int)
df1['sun']=(df1['Dow']==6).astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [29]:
df1

,image_type,account,caption,profile_name,biography,following,likes,posts_count,followers,datetime,...,Dow,Hod,Date,mon,tue,wed,thu,fri,sat,sun
0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,29.0,280.0,8702.0,2021-07-18 09:19:41.056,...,6,9,2021-07-18,0,0,0,0,0,0,1
1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,44.0,280.0,8702.0,2022-03-10 13:45:42.144,...,3,13,2022-03-10,0,0,0,1,0,0,0
2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,209.0,280.0,8702.0,2021-10-20 11:39:44.000,...,2,11,2021-10-20,0,0,1,0,0,0,0
3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,95.0,280.0,8702.0,2021-10-01 12:06:51.136,...,4,12,2021-10-01,0,0,0,0,1,0,0
4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,101.0,280.0,8702.0,2021-07-22 05:00:20.352,...,3,5,2021-07-22,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day 2 Christmas brochure tour Day one brochure...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,10.0,1863.0,2321.0,2021-11-12 09:39:17.632,...,4,9,2021-11-12,0,0,0,0,1,0,0
88510,"['Not Clip', 'Not LineDraw']",sandysfishmongers,#ArmisticeDay #poppy #lestwerespect,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,50.0,1863.0,2321.0,2021-11-11 08:40:42.240,...,3,8,2021-11-11,0,0,0,1,0,0,0
88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,8.0,1863.0,2321.0,2021-11-10 13:14:09.792,...,2,13,2021-11-10,0,0,1,0,0,0,0
88512,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day one brochure tour !! #sandysfishmongers #b...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,9.0,1863.0,2321.0,2021-11-10 13:14:09.792,...,2,13,2021-11-10,0,0,1,0,0,0,0


In [30]:
df = pd.get_dummies(df1, columns=['industry'])

In [31]:
df

,image_type,account,caption,profile_name,biography,following,likes,posts_count,followers,datetime,...,sat,sun,industry_childcare,industry_cosmetics,industry_fashion,industry_fitness,industry_hospitality,industry_real estate,industry_retail,industry_sport & recreation
0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,29.0,280.0,8702.0,2021-07-18 09:19:41.056,...,0,1,0,0,0,0,1,0,0,0
1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,44.0,280.0,8702.0,2022-03-10 13:45:42.144,...,0,0,0,0,0,0,1,0,0,0
2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,209.0,280.0,8702.0,2021-10-20 11:39:44.000,...,0,0,0,0,0,0,1,0,0,0
3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,95.0,280.0,8702.0,2021-10-01 12:06:51.136,...,0,0,0,0,0,0,1,0,0,0
4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,101.0,280.0,8702.0,2021-07-22 05:00:20.352,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day 2 Christmas brochure tour Day one brochure...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,10.0,1863.0,2321.0,2021-11-12 09:39:17.632,...,0,0,0,0,0,0,0,0,1,0
88510,"['Not Clip', 'Not LineDraw']",sandysfishmongers,#ArmisticeDay #poppy #lestwerespect,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,50.0,1863.0,2321.0,2021-11-11 08:40:42.240,...,0,0,0,0,0,0,0,0,1,0
88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,8.0,1863.0,2321.0,2021-11-10 13:14:09.792,...,0,0,0,0,0,0,0,0,1,0
88512,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day one brochure tour !! #sandysfishmongers #b...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,9.0,1863.0,2321.0,2021-11-10 13:14:09.792,...,0,0,0,0,0,0,0,0,1,0


In [32]:
dropped = ['industry_fashion','industry_retail','industry_childcare','industry_fitness','industry_sport & recreation','industry_childcare','industry_real estate','Date','Dow','datetime']
df.drop(dropped,axis=1,inplace=True)
df

,image_type,account,caption,profile_name,biography,following,likes,posts_count,followers,comments,...,Hod,mon,tue,wed,thu,fri,sat,sun,industry_cosmetics,industry_hospitality
0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,29.0,280.0,8702.0,0.0,...,9,0,0,0,0,0,0,1,0,1
1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,44.0,280.0,8702.0,0.0,...,13,0,0,0,1,0,0,0,0,1
2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,209.0,280.0,8702.0,9.0,...,11,0,0,1,0,0,0,0,0,1
3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,95.0,280.0,8702.0,3.0,...,12,0,0,0,0,1,0,0,0,1
4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,101.0,280.0,8702.0,6.0,...,5,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day 2 Christmas brochure tour Day one brochure...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,10.0,1863.0,2321.0,1.0,...,9,0,0,0,0,1,0,0,0,0
88510,"['Not Clip', 'Not LineDraw']",sandysfishmongers,#ArmisticeDay #poppy #lestwerespect,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,50.0,1863.0,2321.0,0.0,...,8,0,0,0,1,0,0,0,0,0
88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,8.0,1863.0,2321.0,0.0,...,13,0,0,1,0,0,0,0,0,0
88512,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day one brochure tour !! #sandysfishmongers #b...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,9.0,1863.0,2321.0,0.0,...,13,0,0,1,0,0,0,0,0,0


## NLP

In [33]:
data=df.astype(str)
def sentiment_analysis(caption):
    def getSubjectivity(text):
        return TextBlob(text).sentiment.subjectivity
  
    #Create a function to get the polarity
    def getPolarity(caption):
        return TextBlob(caption).sentiment.polarity
  
    #Create two new columns 'Subjectivity' & 'Polarity'
    data['TextBlob_Subjectivity'] = data['caption'].apply(getSubjectivity)
    data ['TextBlob_Polarity'] =  data['caption'].apply(getPolarity)
    return df

sentiment_analysis(data['caption'])

,image_type,account,caption,profile_name,biography,following,likes,posts_count,followers,comments,...,Hod,mon,tue,wed,thu,fri,sat,sun,industry_cosmetics,industry_hospitality
0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,29.0,280.0,8702.0,0.0,...,9,0,0,0,0,0,0,1,0,1
1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,44.0,280.0,8702.0,0.0,...,13,0,0,0,1,0,0,0,0,1
2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,209.0,280.0,8702.0,9.0,...,11,0,0,1,0,0,0,0,0,1
3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,95.0,280.0,8702.0,3.0,...,12,0,0,0,0,1,0,0,0,1
4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,101.0,280.0,8702.0,6.0,...,5,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day 2 Christmas brochure tour Day one brochure...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,10.0,1863.0,2321.0,1.0,...,9,0,0,0,0,1,0,0,0,0
88510,"['Not Clip', 'Not LineDraw']",sandysfishmongers,#ArmisticeDay #poppy #lestwerespect,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,50.0,1863.0,2321.0,0.0,...,8,0,0,0,1,0,0,0,0,0
88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,8.0,1863.0,2321.0,0.0,...,13,0,0,1,0,0,0,0,0,0
88512,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day one brochure tour !! #sandysfishmongers #b...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,9.0,1863.0,2321.0,0.0,...,13,0,0,1,0,0,0,0,0,0


In [34]:
hashtag_summary = adv.extract_hashtags(data['caption'])
emoji_summary = adv.extract_emoji(data['caption'])
mention_summary = adv.extract_mentions(data['caption'])

extracted =  (data.assign(hashtags=hashtag_summary['hashtags'],
         hashcounts=hashtag_summary['hashtag_counts'],
         mentions=mention_summary['mentions'],
         mention_count=mention_summary['mention_counts'],
        emoji=emoji_summary['emoji'],
        emoji_text=emoji_summary['emoji_text'],
        emoji_count=emoji_summary['emoji_counts'] 
         ))

extracted.head(5)

,image_type,account,caption,profile_name,biography,following,likes,posts_count,followers,comments,...,industry_hospitality,TextBlob_Subjectivity,TextBlob_Polarity,hashtags,hashcounts,mentions,mention_count,emoji,emoji_text,emoji_count
0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,29.0,280.0,8702.0,0.0,...,1,0.233333,0.200000,[],0,[],0,"[🍕, 🍕, 🍕, 🥐, 🥐, 🥐]","[pizza, pizza, pizza, croissant, croissant, cr...",6
1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,44.0,280.0,8702.0,0.0,...,1,0.454545,0.136364,[],0,[],0,"[😋, 😋]","[face savoring food, face savoring food]",2
2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,209.0,280.0,8702.0,9.0,...,1,0.147222,-0.080556,[],0,[],0,"[🔺, 🔺]","[red triangle pointed up, red triangle pointed...",2
3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,95.0,280.0,8702.0,3.0,...,1,0.477273,0.085227,[],0,[],0,[],[],0
4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,101.0,280.0,8702.0,6.0,...,1,0.444719,0.174416,"[#yeastbakery, #eastlondon, #eastlondonfood, #...",4,[],0,[],[],0


In [35]:
##Hashtag popularity

def captionLength(i):
    return len(i)

caption_length = extracted['caption'].apply(captionLength)
extracted['caption_length'] = caption_length


#Converting the columns into the float datatype
a=['followers']
for i in a:
  extracted[i]= extracted[i].astype(str).astype(float)
extracted



word_freq_hash = adv.word_frequency(extracted['hashtags'].str.join(' '), extracted['followers'].fillna(0))
word_freq_hash.head()
# type(word_freq_hash.word)

d=pd.Series(word_freq_hash.rel_value.values,index=word_freq_hash.word.values ).to_dict()


def countHashtags(tags):
      if len(tags)==0:
        return 0
      sum = 0
      for i in tags:
        sum += (d.get(i))
      return sum/len(tags)

hashtag_average = extracted['hashtags'].apply(countHashtags)
extracted['hashtag_popularity'] = hashtag_average

In [36]:
REPLACE_mentions = re.compile('@[A-Za-z0-9_]+')
REPLACE_hashtags=re.compile('#[A-Za-z0-9_]+')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
GOOD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

try:
    STOPWORDS = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    STOPWORDS = set(stopwords.words('english'))

def lower(text: str) -> str:
    """
    Transforms given text to lower case.
    Example:
    Input: 'I really like New York city'
    Output: 'i really like new your city'
    """

    return text.lower()

def replace_special_characters(text: str) -> str:
    """
    Replaces special characters, such as paranthesis,
    with spacing character
    """

    return REPLACE_BY_SPACE_RE.sub(' ', text)

def replace_br(text: str) -> str:
    """
    Replaces br characters
    """

    return text.replace('br', '')

def filter_out_uncommon_symbols(text: str) -> str:
    """
    Removes any special character that is not in the
    good symbols list (check regular expression)
    """

    return GOOD_SYMBOLS_RE.sub('', text)

def remove_stopwords(text: str) -> str:
    return ' '.join([x for x in text.split() if x and x not in STOPWORDS])


def strip_text(text: str) -> str:
    """
    Removes any left or right spacing (including carriage return) from text.
    Example:
    Input: '  This assignment is cool\n'
    Output: 'This assignment is cool'
    """

    return text.strip()

def remove_brackets_from_list(text:str)->str:
    """
    Remove the [] brackets from the each of the list
    """
    return text

# def translation(text: str) -> str:
  
#     """
#     Replaces any non english words to the english language
#     """
#     translator = Translator()
#     res=translator.translate(text, dest='en')
#     return res.text


def clean_mention(text:str)-> str:
    return REPLACE_mentions.sub(' ', text)

def clean_hashtags(text:str)-> str:
    return REPLACE_hashtags.sub(' ', text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [37]:
PREPROCESSING_PIPELINE = [
                          clean_mention,
                          clean_hashtags,
                          # translation
                          lower,
                          replace_special_characters,
                          replace_br,
                          filter_out_uncommon_symbols,
                          remove_stopwords,
                          strip_text
                          ]

PREPROCESSING_PIPELINE1 = [remove_brackets_from_list]

# Anchor method

def text_prepare1(text: str,filter_methods = None) -> str:
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """
    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE1
    return reduce(lambda txt, f: f(txt), filter_methods, text)

def text_prepare(text: str, filter_methods: List[Callable[[str], str]] = None) -> str:
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """
    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE
    return reduce(lambda txt, f: f(txt), filter_methods, text)

# Pre-processing

print('Pre-processing text...')
print('[Debug] Before:\n{}'.format(extracted.caption[:10]))

L=['hashtags','emoji_text','mentions']
# L=['hashtags','mentions']
A=['caption']

# Replace each sentence with its pre-processed version
extracted['caption'] = extracted['caption'].apply(lambda txt: text_prepare(txt))

for i in L:
  extracted[i]=extracted[i].apply(lambda txt: text_prepare1(str(txt)[1:-1]))

print('[Debug] After:\n{}'.format(extracted.caption[:10]))
print()

print("Pre-processing completed!")


Pre-processing text...
[Debug] Before:
0    It’s our last Pizza Sunday at the arch today. ...
1                         New cakes on our counters 😋😋
2    🔺Opening Times for Half Term🔺: Hi everyone we ...
3    New addition to our selection of sandwiches, o...
4    We are currently closed as we move into our ne...
5    Weekend treats…\nIncluding the return of our “...
6    New vegan “Chocolate & Toasted Almond Cake wit...
7    Posted @withregram • @londoncoffeeshops Pastri...
8    We are looking for experienced part-time baris...
9    Posted @withregram • @marthadelacey Thrilled f...
Name: caption, dtype: object
[Debug] After:
0    last pizza sunday arch today pizzas available ...
1                                   new cakes counters
2    opening times half term hi everyone wanted let...
3    new addition selection sandwiches roasted aube...
4    currently closed move new exciting space aroun...
5    weekend treatsincluding return coconut custard...
6    new vegan chocolate toasted almo

In [38]:
# Hashtags
extracted['hashtag_max'] = extracted.groupby('account')['hashcounts'].transform('max')
extracted['hashtag_min'] = extracted.groupby('account')['hashcounts'].transform('min')
extracted['hashtag_mean'] = extracted.groupby('account')['hashcounts'].transform('mean')
extracted['hashtag_std'] = extracted.groupby('account')['hashcounts'].transform('std')
extracted['biography_length'] = extracted['biography'].str.len()
extracted['profile_name_len'] = extracted['profile_name'].str.len()



In [39]:
extracted.drop('account',1,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [40]:
REPLACE_BY_SPACE_RE = re.compile('[^a-zA-Z_0-9,.]')

def strip_text(text: str) -> str:
    """
    Removes any left or right spacing (including carriage return) from text.
    Example:
    Input: '  This assignment is cool\n'
    Output: 'This assignment is cool'
    """

    return text.strip()


def replace_br(text: str) -> str:
    """
    Replaces br characters
    """

    return text.replace("'", "")


def replace_special_characters(text: str) -> str:
    """
    Replaces special characters, such as paranthesis,
    with spacing character
    """

    return REPLACE_BY_SPACE_RE.sub(' ', text)


from typing import List, Callable, Dict

PREPROCESSING_PIPELINE = [
                          replace_special_characters,
                          strip_text
                          ]



def text_prepare(text: str, filter_methods: List[Callable[[str], str]] = None) -> str:
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """
    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE
    return reduce(lambda txt, f: f(txt), filter_methods, text)

# Pre-processing

print('Pre-processing text...')
print('[Debug] Before:\n{}'.format(extracted.tags[:10]))

L=['tags','confidence_score','accent_color','dominant_colors','bg_color','fore_color']

A=['tags']

for i in L:
  extracted[i]=extracted[i].apply(lambda txt: text_prepare(str(txt)[1:-1]))

print('[Debug] After:\n{}'.format(extracted.tags[:10]))
print()

print("Pre-processing completed!")

Pre-processing text...
[Debug] Before:
0    ['text', 'font', 'graphics', 'screenshot', 'gr...
1    ['dessert', 'baked goods', 'baking', 'snack', ...
2    ['food', 'fruit', 'baked goods', 'breakfast', ...
3    ['food', 'bread', 'fast food', 'bun', 'america...
4    ['text', 'font', 'design', 'yellow', 'graphics...
5    ['snack', 'baked goods', 'baking', 'dessert', ...
6    ['dessert', 'baked goods', 'baking', 'sweetnes...
7    ['text', 'food', 'shelf', 'countertop', 'indoo...
8    ['text', 'table', 'food', 'bakery', 'clothing'...
9    ['baked goods', 'sweetness', 'food', 'baking',...
Name: tags, dtype: object
[Debug] After:
0    text ,  font ,  graphics ,  screenshot ,  grap...
1    dessert ,  baked goods ,  baking ,  snack ,  f...
2    food ,  fruit ,  baked goods ,  breakfast ,  c...
3    food ,  bread ,  fast food ,  bun ,  american ...
4    text ,  font ,  design ,  yellow ,  graphics ,...
5    snack ,  baked goods ,  baking ,  dessert ,  b...
6    dessert ,  baked goods ,  baking , 

In [41]:
extracted['image_type']=extracted['image_type'].apply(lambda txt: text_prepare(str(txt)[1:-1]))

In [42]:
df = pd.get_dummies(extracted, columns=['image_type'])

In [43]:
df

,caption,profile_name,biography,following,likes,posts_count,followers,comments,tags,confidence_score,...,biography_length,profile_name_len,"image_type_Ambiguous Clip , LineDraw","image_type_Ambiguous Clip , Not LineDraw","image_type_Good Clip , LineDraw","image_type_Good Clip , Not LineDraw","image_type_Normal Clip , LineDraw","image_type_Normal Clip , Not LineDraw","image_type_Not Clip , LineDraw","image_type_Not Clip , Not LineDraw"
0,last pizza sunday arch today pizzas available ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,29.0,280.0,8702.0,0.0,"text , font , graphics , screenshot , grap...","0.9980798959732056, 0.9481294751167297, 0.8818...",...,149,12,0,1,0,0,0,0,0,0
1,new cakes counters,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,44.0,280.0,8702.0,0.0,"dessert , baked goods , baking , snack , f...","0.9897554516792297, 0.987897515296936, 0.98287...",...,149,12,0,0,0,0,0,0,0,1
2,opening times half term hi everyone wanted let...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,209.0,280.0,8702.0,9.0,"food , fruit , baked goods , breakfast , c...","0.9808361530303955, 0.9546540379524231, 0.9477...",...,149,12,0,0,0,0,0,0,0,1
3,new addition selection sandwiches roasted aube...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,95.0,280.0,8702.0,3.0,"food , bread , fast food , bun , american ...","0.9958561658859253, 0.981575608253479, 0.97992...",...,149,12,0,0,0,0,0,0,0,1
4,currently closed move new exciting space aroun...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,101.0,280.0,8702.0,6.0,"text , font , design , yellow , graphics ,...","0.9993002414703369, 0.9640201926231384, 0.9513...",...,149,12,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,day 2 christmas ochure tour day one ochure tour #,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,10.0,1863.0,2321.0,1.0,"text , book , screenshot , brochure , prin...","0.9999996423721313, 0.9184167385101318, 0.9103...",...,150,30,0,0,0,0,0,0,0,1
88510,,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,50.0,1863.0,2321.0,0.0,"building , text , electronic signage , chri...","0.9849988222122192, 0.9520490169525146, 0.9216...",...,150,30,0,0,0,0,0,0,0,1
88511,theres one turkey best place buy sandys make c...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,8.0,1863.0,2321.0,0.0,"text , food , snack","0.9999812841415405, 0.9580191373825073, 0.9157...",...,150,30,0,0,0,0,0,0,0,1
88512,day one ochure tour #,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,9.0,1863.0,2321.0,0.0,"text , poster , cartoon","0.9999863505363464, 0.9184243679046631, 0.9104...",...,150,30,0,0,0,0,0,0,0,1


In [44]:
# for i in df1['confidence_score']:

def convert_str2int(i):
  n = []
  y = i.split(', ')
  if len(y[0]) == 0:
    return []
  for j in range(len(y)):
    n.append(float(y[j]))
  return n

df['confidence_score'] = df['confidence_score'].apply(convert_str2int)
df['accent_color'] = df['accent_color'].apply(convert_str2int)
df['dominant_colors'] = df['dominant_colors'].apply(convert_str2int)
df['bg_color'] = df['bg_color'].apply(convert_str2int)
df['fore_color'] = df['fore_color'].apply(convert_str2int)

In [45]:
df

,caption,profile_name,biography,following,likes,posts_count,followers,comments,tags,confidence_score,...,biography_length,profile_name_len,"image_type_Ambiguous Clip , LineDraw","image_type_Ambiguous Clip , Not LineDraw","image_type_Good Clip , LineDraw","image_type_Good Clip , Not LineDraw","image_type_Normal Clip , LineDraw","image_type_Normal Clip , Not LineDraw","image_type_Not Clip , LineDraw","image_type_Not Clip , Not LineDraw"
0,last pizza sunday arch today pizzas available ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,29.0,280.0,8702.0,0.0,"text , font , graphics , screenshot , grap...","[0.9980798959732056, 0.9481294751167297, 0.881...",...,149,12,0,1,0,0,0,0,0,0
1,new cakes counters,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,44.0,280.0,8702.0,0.0,"dessert , baked goods , baking , snack , f...","[0.9897554516792297, 0.987897515296936, 0.9828...",...,149,12,0,0,0,0,0,0,0,1
2,opening times half term hi everyone wanted let...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,209.0,280.0,8702.0,9.0,"food , fruit , baked goods , breakfast , c...","[0.9808361530303955, 0.9546540379524231, 0.947...",...,149,12,0,0,0,0,0,0,0,1
3,new addition selection sandwiches roasted aube...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,95.0,280.0,8702.0,3.0,"food , bread , fast food , bun , american ...","[0.9958561658859253, 0.981575608253479, 0.9799...",...,149,12,0,0,0,0,0,0,0,1
4,currently closed move new exciting space aroun...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,101.0,280.0,8702.0,6.0,"text , font , design , yellow , graphics ,...","[0.9993002414703369, 0.9640201926231384, 0.951...",...,149,12,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,day 2 christmas ochure tour day one ochure tour #,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,10.0,1863.0,2321.0,1.0,"text , book , screenshot , brochure , prin...","[0.9999996423721313, 0.9184167385101318, 0.910...",...,150,30,0,0,0,0,0,0,0,1
88510,,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,50.0,1863.0,2321.0,0.0,"building , text , electronic signage , chri...","[0.9849988222122192, 0.9520490169525146, 0.921...",...,150,30,0,0,0,0,0,0,0,1
88511,theres one turkey best place buy sandys make c...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,8.0,1863.0,2321.0,0.0,"text , food , snack","[0.9999812841415405, 0.9580191373825073, 0.915...",...,150,30,0,0,0,0,0,0,0,1
88512,day one ochure tour #,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,9.0,1863.0,2321.0,0.0,"text , poster , cartoon","[0.9999863505363464, 0.9184243679046631, 0.910...",...,150,30,0,0,0,0,0,0,0,1


In [46]:
def convert_str2list(i):
  n = []
  y = i.split(', ')
  for j in range(len(y)):
    n.append(str(y[j]))
  return n
df['tags'] = df['tags'].apply(convert_str2list)
df['tags'][0]

['text ',
 ' font ',
 ' graphics ',
 ' screenshot ',
 ' graphic design ',
 ' design ',
 ' typography']

In [47]:
df['confidence_score'][6]

[0.9938297271728516,
 0.9784528613090515,
 0.977806806564331,
 0.966992974281311,
 0.9616279602050781,
 0.9515609741210938,
 0.9280351400375366,
 0.912075936794281,
 0.9083282947540283,
 0.9079588651657104,
 0.8951475620269775,
 0.8874362707138062,
 0.8836393356323242,
 0.8746436834335327,
 0.8713182210922241,
 0.8668385744094849,
 0.8426526784896851,
 0.786042332649231,
 0.5681669116020203]

In [48]:
def trim_lowConfidence_tags(tag_column):
  for scoreList in df['confidence_score']:
    for score_index in range(len(scoreList)):
      if scoreList[score_index]<0.5:
        return tag_column[:score_index]

df['tags'] = df['tags'].apply(trim_lowConfidence_tags)  

In [49]:
df.drop('confidence_score',1,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [50]:
df.dominant_colors[1]

[0.6470588235294118, 0.16470588235294117, 0.16470588235294117]

In [51]:
df['dominant_colors'][:3]

0         [1.0, 0.7529411764705882, 0.796078431372549]
1    [0.6470588235294118, 0.16470588235294117, 0.16...
2    [0.6470588235294118, 0.16470588235294117, 0.16...
Name: dominant_colors, dtype: object

In [52]:
def separate_column(column):
  if len(column)>0:
    return column[0]
  else:
    return []

df['dominant_colors_R'] = df['dominant_colors'].apply(separate_column)  
df['accent_color_R'] = df['accent_color'].apply(separate_column)  
df['bg_color_R'] = df['bg_color'].apply(separate_column)  
df['fore_color_R'] = df['fore_color'].apply(separate_column)  

In [53]:
df

,caption,profile_name,biography,following,likes,posts_count,followers,comments,tags,accent_color,...,"image_type_Good Clip , LineDraw","image_type_Good Clip , Not LineDraw","image_type_Normal Clip , LineDraw","image_type_Normal Clip , Not LineDraw","image_type_Not Clip , LineDraw","image_type_Not Clip , Not LineDraw",dominant_colors_R,accent_color_R,bg_color_R,fore_color_R
0,last pizza sunday arch today pizzas available ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,29.0,280.0,8702.0,0.0,"[text , font , graphics , screenshot , gra...","[0.788235294117647, 0.00784313725490196, 0.007...",...,0,0,0,0,0,0,1.000000,0.788235,1.000000,1.000000
1,new cakes counters,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,44.0,280.0,8702.0,0.0,"[dessert , baked goods , baking , snack , ...","[0.6901960784313725, 0.14901960784313725, 0.10...",...,0,0,0,0,0,1,0.647059,0.690196,0.647059,0.000000
2,opening times half term hi everyone wanted let...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,209.0,280.0,8702.0,9.0,"[food , fruit , baked goods , breakfast , ...","[0.6588235294117647, 0.4392156862745098, 0.141...",...,0,0,0,0,0,1,0.647059,0.658824,0.647059,0.647059
3,new addition selection sandwiches roasted aube...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,95.0,280.0,8702.0,3.0,"[food , bread , fast food , bun , american...","[0.24313725490196078, 0.12549019607843137, 0.0...",...,0,0,0,0,0,1,0.647059,0.243137,0.647059,0.647059
4,currently closed move new exciting space aroun...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,101.0,280.0,8702.0,6.0,"[text , font , design , yellow , graphics ...","[0.792156862745098, 0.7372549019607844, 0.0039...",...,0,0,0,0,0,0,1.000000,0.792157,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,day 2 christmas ochure tour day one ochure tour #,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,10.0,1863.0,2321.0,1.0,"[text , book , screenshot , brochure , pri...","[0.011764705882352941, 0.5019607843137255, 0.7...",...,0,0,0,0,0,1,1.000000,0.011765,0.000000,1.000000
88510,,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,50.0,1863.0,2321.0,0.0,"[building , text , electronic signage , chr...","[0.6470588235294118, 0.1568627450980392, 0.149...",...,0,0,0,0,0,1,0.501961,0.647059,0.000000,1.000000
88511,theres one turkey best place buy sandys make c...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,8.0,1863.0,2321.0,0.0,"[text , food , snack]","[0.6588235294117647, 0.00784313725490196, 0.71...",...,0,0,0,0,0,1,0.501961,0.658824,0.501961,0.501961
88512,day one ochure tour #,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,9.0,1863.0,2321.0,0.0,"[text , poster , cartoon]","[0.01568627450980392, 0.5098039215686274, 0.78...",...,0,0,0,0,0,1,1.000000,0.015686,1.000000,1.000000


In [54]:
dropped = ['accent_color', 'dominant_colors', 'bg_color','fore_color']
data = df.drop(dropped,axis=1)

In [55]:
data

,caption,profile_name,biography,following,likes,posts_count,followers,comments,tags,is_bw,...,"image_type_Good Clip , LineDraw","image_type_Good Clip , Not LineDraw","image_type_Normal Clip , LineDraw","image_type_Normal Clip , Not LineDraw","image_type_Not Clip , LineDraw","image_type_Not Clip , Not LineDraw",dominant_colors_R,accent_color_R,bg_color_R,fore_color_R
0,last pizza sunday arch today pizzas available ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,29.0,280.0,8702.0,0.0,"[text , font , graphics , screenshot , gra...",0,...,0,0,0,0,0,0,1.000000,0.788235,1.000000,1.000000
1,new cakes counters,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,44.0,280.0,8702.0,0.0,"[dessert , baked goods , baking , snack , ...",0,...,0,0,0,0,0,1,0.647059,0.690196,0.647059,0.000000
2,opening times half term hi everyone wanted let...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,209.0,280.0,8702.0,9.0,"[food , fruit , baked goods , breakfast , ...",0,...,0,0,0,0,0,1,0.647059,0.658824,0.647059,0.647059
3,new addition selection sandwiches roasted aube...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,95.0,280.0,8702.0,3.0,"[food , bread , fast food , bun , american...",0,...,0,0,0,0,0,1,0.647059,0.243137,0.647059,0.647059
4,currently closed move new exciting space aroun...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,101.0,280.0,8702.0,6.0,"[text , font , design , yellow , graphics ...",0,...,0,0,0,0,0,0,1.000000,0.792157,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,day 2 christmas ochure tour day one ochure tour #,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,10.0,1863.0,2321.0,1.0,"[text , book , screenshot , brochure , pri...",0,...,0,0,0,0,0,1,1.000000,0.011765,0.000000,1.000000
88510,,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,50.0,1863.0,2321.0,0.0,"[building , text , electronic signage , chr...",0,...,0,0,0,0,0,1,0.501961,0.647059,0.000000,1.000000
88511,theres one turkey best place buy sandys make c...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,8.0,1863.0,2321.0,0.0,"[text , food , snack]",0,...,0,0,0,0,0,1,0.501961,0.658824,0.501961,0.501961
88512,day one ochure tour #,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,9.0,1863.0,2321.0,0.0,"[text , poster , cartoon]",0,...,0,0,0,0,0,1,1.000000,0.015686,1.000000,1.000000


##FastText

### Image Tags

In [56]:
data['tags']=data['tags'].apply(lambda txt: text_prepare(str(txt)[1:-1]))
tokenized_tag = data['tags'].apply(lambda x: x.split()[1:-1])

In [57]:
modelFastTextSkipGram = FastText(tokenized_tag, 
                     size=50, # desired no. of features/independent variables
                     window=2,  # context window size
                     min_count=2, # Ignores all words with total frequency lower than 2.  
                     workers=32, # no.of cores
                     hs = 0,
                     negative = 10, # for negative sampling
                     sg=1  # 1 for Skipgram model
                     )

modelFastTextSkipGram.build_vocab(tokenized_tag, update=True)
modelFastTextSkipGram.train(tokenized_tag, total_examples= len(tokenized_tag), epochs=2)

2022-05-22 23:29:12,935 | WARNING | base_any2vec.py:723 | __init__ | consider setting layer size to a multiple of 4 for greater performance
2022-05-22 23:29:12,936 | INFO | word2vec.py:1567 | scan_vocab | collecting all words and their counts
2022-05-22 23:29:12,947 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-22 23:29:12,981 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #10000, processed 184916 words, keeping 2209 word types
2022-05-22 23:29:13,023 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #20000, processed 375164 words, keeping 2671 word types
2022-05-22 23:29:13,062 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #30000, processed 562939 words, keeping 2938 word types
2022-05-22 23:29:13,101 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #40000, processed 754746 words, keeping 3128 word types
2022-05-22 23:29:13,138 | INFO | word2vec.py:1552

In [58]:
modelFastTextSkipGram.wv.most_similar("Gastroenteritis")

2022-05-22 23:29:53,564 | INFO | keyedvectors.py:1360 | init_sims | precomputing L2-norms of word weight vectors
2022-05-22 23:29:53,567 | INFO | keyedvectors.py:2011 | init_sims | precomputing L2-norms of ngram weight vectors


[('phalaenopsis', 0.875116765499115),
 ('analgesic', 0.8542455434799194),
 ('tagliatelle', 0.8494495749473572),
 ('bandage', 0.8434736132621765),
 ('salchicha3n', 0.8409441709518433),
 ('alps', 0.8372900485992432),
 ('baarnaise', 0.8331668972969055),
 ('gazpacho', 0.8266377449035645),
 ('cdj', 0.8259006142616272),
 ('vinyl', 0.8249576091766357)]

In [59]:
modelFastTextSkipGram.wv.most_similar("pizza")

[('flatbread', 0.8586891293525696),
 ('italian', 0.8420588374137878),
 ('sicilian', 0.8088645339012146),
 ('california', 0.80636066198349),
 ('pepperoni', 0.7982450127601624),
 ('style', 0.7957154512405396),
 ('cheese', 0.7772330641746521),
 ('manakish', 0.7543833255767822),
 ('cheddar', 0.7449068427085876),
 ('cheesemaking', 0.7388368844985962)]

In [60]:
def word_vector(tokens, size,model):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [61]:
fastText_arrays = np.zeros((len(tokenized_tag), 50)) 
for i in range(len(tokenized_tag)):
    fastText_arrays[i,:] = word_vector(tokenized_tag[i], 50,modelFastTextSkipGram)
fastText_df = pd.DataFrame(fastText_arrays)
fastText_df.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(88514, 50)

In [62]:
type(data)

pandas.core.frame.DataFrame

### caption

In [63]:
tokenized_caption = data['caption'].apply(lambda x: str(x).split()) # tokenizing
modelFastTextSkipGramCaption = FastText(tokenized_caption, 
                     size=50, # desired no. of features/independent variables
                     window=2,  # context window size
                     min_count=2, # Ignores all words with total frequency lower than 2.  
                     workers=32, # no.of cores
                     hs = 0,
                     negative = 10, # for negative sampling
                     sg=1  # 1 for Skipgram model
                     )

modelFastTextSkipGramCaption.build_vocab(tokenized_caption, update=True)
modelFastTextSkipGramCaption.train(tokenized_caption, total_examples= len(tokenized_caption), epochs=20)

2022-05-22 23:30:14,676 | WARNING | base_any2vec.py:723 | __init__ | consider setting layer size to a multiple of 4 for greater performance
2022-05-22 23:30:14,678 | INFO | word2vec.py:1567 | scan_vocab | collecting all words and their counts
2022-05-22 23:30:14,685 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-22 23:30:14,739 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #10000, processed 267626 words, keeping 27703 word types
2022-05-22 23:30:14,799 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #20000, processed 532556 words, keeping 43036 word types
2022-05-22 23:30:14,869 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #30000, processed 831440 words, keeping 57740 word types
2022-05-22 23:30:14,936 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #40000, processed 1099425 words, keeping 69288 word types
2022-05-22 23:30:15,000 | INFO | word2vec.py

In [64]:
wordvec_arrays = np.zeros((len(tokenized_caption), 50)) 
for i in range(len(tokenized_caption)):
    wordvec_arrays[i,:] = word_vector(tokenized_caption[i], 50,modelFastTextSkipGramCaption)
wordvec_caption = pd.DataFrame(wordvec_arrays)
wordvec_caption.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(88514, 50)

### hashtags

In [65]:
tokenized_hastags = df['hashtags'].apply(lambda x: str(x).split()) # tokenizing
modelFastTextSkipGramHastags = FastText(tokenized_hastags, 
                     size=50, # desired no. of features/independent variables
                     window=2,  # context window size
                     min_count=2, # Ignores all words with total frequency lower than 2.  
                     workers=32, # no.of cores
                     hs = 0,
                     negative = 10, # for negative sampling
                     sg=0  # 0 for CBOW model
                     )

modelFastTextSkipGramHastags.build_vocab(tokenized_hastags, update=True)
modelFastTextSkipGramHastags.train(tokenized_hastags, total_examples= len(tokenized_hastags), epochs=20)

2022-05-22 23:39:01,783 | WARNING | base_any2vec.py:723 | __init__ | consider setting layer size to a multiple of 4 for greater performance
2022-05-22 23:39:01,785 | INFO | word2vec.py:1567 | scan_vocab | collecting all words and their counts
2022-05-22 23:39:01,786 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-22 23:39:01,805 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #10000, processed 50969 words, keeping 10996 word types
2022-05-22 23:39:01,825 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #20000, processed 113744 words, keeping 21000 word types
2022-05-22 23:39:01,842 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #30000, processed 171307 words, keeping 28787 word types
2022-05-22 23:39:01,868 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #40000, processed 238519 words, keeping 35303 word types
2022-05-22 23:39:01,891 | INFO | word2vec.py:1

In [66]:
wordvec_arrays = np.zeros((len(tokenized_hastags), 50)) 
for i in range(len(tokenized_hastags)):
    wordvec_arrays[i,:] = word_vector(tokenized_hastags[i], 50,modelFastTextSkipGramHastags)
wordvec_Hashtag = pd.DataFrame(wordvec_arrays)
wordvec_Hashtag.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(88514, 50)

### emojiText

In [67]:
tokenized_emojiText = df['hashtags'].apply(lambda x: str(x).split()) # tokenizing
modelFastTextSkipGramEmojiText = FastText(tokenized_emojiText, 
                     size=50, # desired no. of features/independent variables
                     window=2,  # context window size
                     min_count=2, # Ignores all words with total frequency lower than 2.  
                     workers=32, # no.of cores
                     hs = 0,
                     negative = 10, # for negative sampling
                     sg=1  # 1 for Skipgram model
                     )

modelFastTextSkipGramEmojiText.build_vocab(tokenized_emojiText, update=True)
modelFastTextSkipGramEmojiText.train(tokenized_emojiText, total_examples= len(tokenized_emojiText), epochs=20)

2022-05-22 23:41:22,653 | WARNING | base_any2vec.py:723 | __init__ | consider setting layer size to a multiple of 4 for greater performance
2022-05-22 23:41:22,655 | INFO | word2vec.py:1567 | scan_vocab | collecting all words and their counts
2022-05-22 23:41:22,656 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-22 23:41:22,677 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #10000, processed 50969 words, keeping 10996 word types
2022-05-22 23:41:22,694 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #20000, processed 113744 words, keeping 21000 word types
2022-05-22 23:41:22,714 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #30000, processed 171307 words, keeping 28787 word types
2022-05-22 23:41:22,732 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #40000, processed 238519 words, keeping 35303 word types
2022-05-22 23:41:22,753 | INFO | word2vec.py:1

In [68]:
wordvec_arrays = np.zeros((len(tokenized_emojiText), 50)) 
for i in range(len(tokenized_emojiText)):
    wordvec_arrays[i,:] = word_vector(tokenized_emojiText[i], 50,modelFastTextSkipGramEmojiText)
wordvec_emojiText = pd.DataFrame(wordvec_arrays)
wordvec_emojiText.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(88514, 50)

### mentions

In [69]:
tokenized_mentions = df['mentions'].apply(lambda x: str(x).split()) # tokenizing
modelFastTextSkipGramMentions = FastText(tokenized_mentions, 
                     size=50, # desired no. of features/independent variables
                     window=2,  # context window size
                     min_count=2, # Ignores all words with total frequency lower than 2.  
                     workers=32, # no.of cores
                     hs = 0,
                     negative = 10, # for negative sampling
                     sg=1  # 1 for Skipgram model
                     )

modelFastTextSkipGramMentions.build_vocab(tokenized_mentions, update=True)
modelFastTextSkipGramMentions.train(tokenized_mentions, total_examples= len(tokenized_mentions), epochs=20)

2022-05-22 23:44:04,012 | WARNING | base_any2vec.py:723 | __init__ | consider setting layer size to a multiple of 4 for greater performance
2022-05-22 23:44:04,014 | INFO | word2vec.py:1567 | scan_vocab | collecting all words and their counts
2022-05-22 23:44:04,015 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-22 23:44:04,024 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #10000, processed 4754 words, keeping 2945 word types
2022-05-22 23:44:04,032 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #20000, processed 9509 words, keeping 5724 word types
2022-05-22 23:44:04,041 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #30000, processed 14594 words, keeping 8486 word types
2022-05-22 23:44:04,047 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #40000, processed 19425 words, keeping 10973 word types
2022-05-22 23:44:04,055 | INFO | word2vec.py:1552 | _s

In [70]:
wordvec_arrays = np.zeros((len(tokenized_mentions), 50)) 
for i in range(len(tokenized_mentions)):
    wordvec_arrays[i,:] = word_vector(tokenized_mentions[i], 50,modelFastTextSkipGramMentions)
wordvec_Mentions = pd.DataFrame(wordvec_arrays)
wordvec_Mentions.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(88514, 50)

In [91]:
dataset_fastText2 = pd.DataFrame(np.hstack((fastText_df,wordvec_caption,wordvec_Hashtag,wordvec_emojiText,wordvec_Mentions,data)))

In [92]:
dataset_fastText2[254]

0         29.0
1         44.0
2        209.0
3         95.0
4        101.0
         ...  
88509     10.0
88510     50.0
88511      8.0
88512      9.0
88513     11.0
Name: 254, Length: 88514, dtype: object

In [95]:
print(len(dataset_fastText2.columns))
a=len(dataset_fastText2.columns)-45
print(a)
dataset_fastText2[len(dataset_fastText2.columns)-43]
dataset_fastText2.rename(columns = {a:'likes'}, inplace = True)
df=removeColumnContainString(dataset_fastText2)

299
254
Couldn't covert 250 to float
Couldn't covert 251 to float
Couldn't covert 252 to float
Couldn't covert 258 to float
Couldn't covert 272 to float
Couldn't covert 274 to float
Couldn't covert 276 to float
Couldn't covert 277 to float


In [96]:
dataset_fastText2 = dataset_fastText2.reset_index()

In [99]:
# dataset_fastText2[254]

In [100]:
dropped = [250,251,252,258,272,274,276,277]
dataset_fastText2.drop(dropped,axis=1,inplace=True)
dataset_fastText2

,index,0,1,2,3,4,5,6,7,8,...,289,290,291,292,293,294,295,296,297,298
0,0,0.276721,0.113139,0.034563,0.134659,-0.546147,-0.182426,-0.160515,-0.59871,-0.025352,...,0,0,0,0,0,0,1.0,0.788235,1.0,1.0
1,1,0.048888,0.576153,-0.355953,-0.162595,-0.233754,0.050176,-0.115882,-0.507439,-0.257501,...,0,0,0,0,0,1,0.647059,0.690196,0.647059,0.0
2,2,-0.01117,0.387848,-0.242469,-0.23539,-0.023971,0.032928,-0.171692,-0.523088,-0.063837,...,0,0,0,0,0,1,0.647059,0.658824,0.647059,0.647059
3,3,0.038574,0.280561,-0.386209,-0.046993,-0.029427,0.042889,0.047218,-0.454082,0.008513,...,0,0,0,0,0,1,0.647059,0.243137,0.647059,0.647059
4,4,0.300424,0.106753,0.064463,0.109077,-0.548251,-0.159049,-0.23577,-0.599744,-0.023256,...,0,0,0,0,0,0,1.0,0.792157,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,88509,0.286842,0.243731,-0.033131,-0.079824,-0.226685,-0.259312,-0.10287,-0.364772,0.205498,...,0,0,0,0,0,1,1.0,0.011765,0.0,1.0
88510,88510,0.381816,0.00839,0.00374,-0.275623,-0.553979,-0.210371,-0.156245,-0.342969,0.01574,...,0,0,0,0,0,1,0.501961,0.647059,0.0,1.0
88511,88511,-0.019925,0.451035,-0.110425,-0.269043,-0.002999,0.007781,0.010109,-0.355132,0.236052,...,0,0,0,0,0,1,0.501961,0.658824,0.501961,0.501961
88512,88512,-0.052274,0.105286,-0.057282,-0.22315,-0.353306,-0.000037,-0.157015,-0.686578,0.213414,...,0,0,0,0,0,1,1.0,0.015686,1.0,1.0


In [101]:
from utils import clean_dataset
clean_dataset(dataset_fastText2)

,index,0,1,2,3,4,5,6,7,8,...,289,290,291,292,293,294,295,296,297,298
0,0.0,0.276721,0.113139,0.034563,0.134659,-0.546147,-0.182426,-0.160515,-0.598710,-0.025352,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.788235,1.000000,1.000000
1,1.0,0.048888,0.576153,-0.355953,-0.162595,-0.233754,0.050176,-0.115882,-0.507439,-0.257501,...,0.0,0.0,0.0,0.0,0.0,1.0,0.647059,0.690196,0.647059,0.000000
2,2.0,-0.011170,0.387848,-0.242469,-0.235390,-0.023971,0.032928,-0.171692,-0.523088,-0.063837,...,0.0,0.0,0.0,0.0,0.0,1.0,0.647059,0.658824,0.647059,0.647059
3,3.0,0.038574,0.280561,-0.386209,-0.046993,-0.029427,0.042889,0.047218,-0.454082,0.008513,...,0.0,0.0,0.0,0.0,0.0,1.0,0.647059,0.243137,0.647059,0.647059
4,4.0,0.300424,0.106753,0.064463,0.109077,-0.548251,-0.159049,-0.235770,-0.599744,-0.023256,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.792157,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,88509.0,0.286842,0.243731,-0.033131,-0.079824,-0.226685,-0.259312,-0.102870,-0.364772,0.205498,...,0.0,0.0,0.0,0.0,0.0,1.0,1.000000,0.011765,0.000000,1.000000
88510,88510.0,0.381816,0.008390,0.003740,-0.275623,-0.553979,-0.210371,-0.156245,-0.342969,0.015740,...,0.0,0.0,0.0,0.0,0.0,1.0,0.501961,0.647059,0.000000,1.000000
88511,88511.0,-0.019925,0.451035,-0.110425,-0.269043,-0.002999,0.007781,0.010109,-0.355132,0.236052,...,0.0,0.0,0.0,0.0,0.0,1.0,0.501961,0.658824,0.501961,0.501961
88512,88512.0,-0.052274,0.105286,-0.057282,-0.223150,-0.353306,-0.000037,-0.157015,-0.686578,0.213414,...,0.0,0.0,0.0,0.0,0.0,1.0,1.000000,0.015686,1.000000,1.000000


## MinMaxScaling

In [102]:
from sklearn import preprocessing
df = pd.DataFrame(dataset_fastText2)
dataset_fastText33=df.astype(str)

normalizer = preprocessing.MinMaxScaler(feature_range=(0, 1))
normalized_df = pd.DataFrame(normalizer.fit_transform(dataset_fastText33),  columns = dataset_fastText33.columns)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [103]:
y=normalized_df['likes']                #Target Variable 
X = normalized_df.drop('likes', axis=1) #Feature Matrix 

In [104]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)

### LightLGBM

In [105]:
from lightgbm import LGBMRegressor
model_lgb = LGBMRegressor(colsample_bytree=0.8,learning_rate=0.01, max_depth=8,min_child_weight=1, min_split_gain=0.0222415, n_estimators=35000,num_leaves=966, reg_alpha=0.04, reg_lambda=0.073,subsample=0.6)

In [106]:
start = datetime.datetime.now()
lgbm = model_lgb.fit(x_train, y_train,eval_set=[(x_train, y_train), (x_test, y_test)],eval_metric ='mae')
end = datetime.datetime.now()
end-start

Streaming output truncated to the last 5000 lines.
[30001]	training's l2: 5.46978e-05	training's l1: 0.00176364	valid_1's l2: 6.74843e-05	valid_1's l1: 0.00177464
[30002]	training's l2: 5.46978e-05	training's l1: 0.00176364	valid_1's l2: 6.74843e-05	valid_1's l1: 0.00177464
[30003]	training's l2: 5.46978e-05	training's l1: 0.00176364	valid_1's l2: 6.74843e-05	valid_1's l1: 0.00177464
[30004]	training's l2: 5.46978e-05	training's l1: 0.00176364	valid_1's l2: 6.74843e-05	valid_1's l1: 0.00177464
[30005]	training's l2: 5.46978e-05	training's l1: 0.00176364	valid_1's l2: 6.74843e-05	valid_1's l1: 0.00177464
[30006]	training's l2: 5.46978e-05	training's l1: 0.00176364	valid_1's l2: 6.74843e-05	valid_1's l1: 0.00177464
[30007]	training's l2: 5.46978e-05	training's l1: 0.00176364	valid_1's l2: 6.74843e-05	valid_1's l1: 0.00177464
[30008]	training's l2: 5.46978e-05	training's l1: 0.00176364	valid_1's l2: 6.74843e-05	valid_1's l1: 0.00177464
[30009]	training's l2: 5.46978e-05	training's l1: 0.0

datetime.timedelta(seconds=683, microseconds=251718)

In [107]:
y_pred_lgbm = lgbm.predict(x_test)
lgbm.score(x_test, y_test)

0.5677903903005823

In [108]:
rmse,R2,MAE,MSE,RMSLE,max_error_= metrics(y_test,y_pred_lgbm)
print(R2)

0.5677903903005823


In [109]:
dict_info = {'Model': [], 'RMSE': [], 'R2': [],'MAE': [], 'RMSLE': [],'max_error_':[]}

rmse,R2,MAE,MSE,RMSLE,max_error_= metrics(y_test,y_pred_lgbm)

dict_info['Model'].append('Lightgbm')
dict_info['RMSE'].append(rmse)
dict_info['R2'].append(R2)
dict_info['MAE'].append(MAE)
dict_info['RMSLE'].append(RMSLE)
dict_info['max_error_'].append(max_error_)

###XGBoost model

In [110]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
        enable_categorical=True,
        objective = 'reg:squarederror',
        colsample_bytree = 0.5,
        learning_rate = 0.05,
        max_depth = 6,
        min_child_weight = 1,
        n_estimators = 1000,
        subsample = 0.7)

xgb_model.fit(x_train, y_train,eval_set=[(x_train, y_train), (x_test, y_test)], eval_metric='mae')

[0]	validation_0-mae:0.471931	validation_1-mae:0.471954
[1]	validation_0-mae:0.448341	validation_1-mae:0.448359
[2]	validation_0-mae:0.425928	validation_1-mae:0.425947
[3]	validation_0-mae:0.404635	validation_1-mae:0.404651
[4]	validation_0-mae:0.384407	validation_1-mae:0.384424
[5]	validation_0-mae:0.365191	validation_1-mae:0.365208
[6]	validation_0-mae:0.346936	validation_1-mae:0.346958
[7]	validation_0-mae:0.329592	validation_1-mae:0.329618
[8]	validation_0-mae:0.313117	validation_1-mae:0.313148
[9]	validation_0-mae:0.297464	validation_1-mae:0.297494
[10]	validation_0-mae:0.282596	validation_1-mae:0.282624
[11]	validation_0-mae:0.268471	validation_1-mae:0.2685
[12]	validation_0-mae:0.255052	validation_1-mae:0.25508
[13]	validation_0-mae:0.242304	validation_1-mae:0.242333
[14]	validation_0-mae:0.230194	validation_1-mae:0.230222
[15]	validation_0-mae:0.218687	validation_1-mae:0.218718
[16]	validation_0-mae:0.207757	validation_1-mae:0.207787
[17]	validation_0-mae:0.197372	validation_1-

XGBRegressor(colsample_bytree=0.5, enable_categorical=True, learning_rate=0.05,
             max_depth=6, n_estimators=1000, objective='reg:squarederror',
             subsample=0.7)

In [111]:
y_pred_W2vec_xgboost = xgb_model.predict(x_test)
xgb_model.score(x_test, y_test)

0.6484845265831959

In [112]:
xgb_model.score(x_test, y_test)

0.6484845265831959

In [113]:
rmse,R2,MAE,MSE,RMSLE,max_error_= metrics(y_test,y_pred_W2vec_xgboost)

dict_info['Model'].append('XGboost')
dict_info['RMSE'].append(rmse)
dict_info['R2'].append(R2)
dict_info['MAE'].append(MAE)
dict_info['RMSLE'].append(RMSLE)
dict_info['max_error_'].append(max_error_)

## Random Forest

In [114]:
# utils.hyperparameterTuning_RandomForest(x_train_w2v_df,y_train_w2v_df)
rnd_clf = RandomForestRegressor(n_estimators=20, min_samples_split=10, min_samples_leaf=4,max_features='sqrt',max_depth=10,bootstrap=True)
rnd_clf.fit(x_train, y_train)
y_pred_w2vec_rnd = rnd_clf.predict(x_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [115]:
rmse,R2,MAE,MSE,RMSLE,max_error_= metrics(y_test,y_pred_w2vec_rnd)
print(R2)

dict_info['Model'].append('Random Forest')
dict_info['RMSE'].append(rmse)
dict_info['R2'].append(R2)
dict_info['MAE'].append(MAE)
dict_info['RMSLE'].append(RMSLE)
dict_info['max_error_'].append(max_error_)

0.4384221921648912


## MLP

In [116]:
from sklearn.neural_network import MLPRegressor
mlp_GS = MLPRegressor(activation='relu',alpha=0.001,hidden_layer_sizes=(50, 50, 50),learning_rate='constant',solver='adam')
mlp_GS.fit(x_train, y_train)
y_pred_w2vec_mlp = mlp_GS.predict(x_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [117]:
rmse,R2,MAE,MSE,RMSLE,max_error_= metrics(y_test,y_pred_w2vec_mlp)

dict_info['Model'].append('MLP')
dict_info['RMSE'].append(rmse)
dict_info['R2'].append(R2)
dict_info['MAE'].append(MAE)
dict_info['RMSLE'].append(RMSLE)
dict_info['max_error_'].append(max_error_)

## SVC

In [118]:
from sklearn.svm import LinearSVR
svr= LinearSVR(C=1.0, dual=True, epsilon=1.5, fit_intercept=True,intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,tol=0.0001, verbose=0)
svr.fit(x_train, y_train)
y_pred_w2vec_SVR = svr.predict(x_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [119]:
rmse,R2,MAE,MSE,RMSLE,max_error_= metrics(y_test,y_pred_w2vec_SVR)

dict_info['Model'].append('SVR')
dict_info['RMSE'].append(rmse)
dict_info['R2'].append(R2)
dict_info['MAE'].append(MAE)
dict_info['RMSLE'].append(RMSLE)
dict_info['max_error_'].append(max_error_)

In [120]:
dfData = pd.DataFrame(dict_info)
dfData

,Model,RMSE,R2,MAE,RMSLE,max_error_
0,Lightgbm,0.008215,0.567790,0.001775,-4.801808,0.525262
1,XGboost,0.007408,0.648485,0.001338,-4.905136,0.528714
2,Random Forest,0.009364,0.438422,0.002144,-4.670888,0.595959
3,MLP,0.010070,0.350565,0.003266,-4.598211,0.566379
4,SVR,0.012907,-0.066904,0.003232,-4.350005,0.639221
